In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

In [2]:
#    Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
#    Spark Streaming
from pyspark.streaming import StreamingContext
#    Kafka
from pyspark.streaming.kafka import KafkaUtils
#    json parsing
import json

In [3]:
conf = SparkConf()
conf.set('spark.driver.host', os.environ['SERVER_HOSTNAME'])
conf.set('spark.driver.port', os.environ['SPARK_PORT_1'])
conf.set('spark.blockManager.port', os.environ['SPARK_PORT_2'])
conf.set('spark.ui.port', os.environ['SPARK_PORT_3'])
conf.set('spark.master', 'yarn')
conf.set('spark.authenticate', True)
conf.set('spark.network.crypto.enabled', True)
conf.set('spark.authenticate.enableSaslEncryption', True)
sc = SparkContext(conf = conf)
spark = SparkSession(sc)
sc.setLogLevel("WARN")

In [4]:
from confluent.schemaregistry.client import CachedSchemaRegistryClient
from confluent.schemaregistry.serializers import MessageSerializer
schema_registry_client = CachedSchemaRegistryClient(url='http://sstreaming.cern.ch:8081')
serializer = MessageSerializer(schema_registry_client)


# simple decode to replace Kafka-streaming's built-in decode decoding UTF8 ()
def decoder(s):
    decoded_message = serializer.decode_message(s)
    return decoded_message

ImportError: No module named confluent.schemaregistry.client

In [ ]:
ssc = StreamingContext(sc, 60)

In [ ]:
kafkaStream = KafkaUtils.createStream(ssc, 'sstreaming:2181', 'spark-streaming', {'twitter':1}, valueDecoder=decoder)

In [8]:
parsed = kafkaStream.map(lambda v: v[1])

In [8]:
kafkaStream.count().map(lambda x:'Tweets in this batch: %s' % x).pprint()

In [10]:
authors_dstream = parsed.map(lambda tweet: tweet['user']['screen_name'])

In [9]:
ssc.start()
ssc.awaitTermination(timeout=180)

-------------------------------------------
Time: 2017-11-22 14:03:00
-------------------------------------------

-------------------------------------------
Time: 2017-11-22 14:04:00
-------------------------------------------

-------------------------------------------
Time: 2017-11-22 14:05:00
-------------------------------------------



In [11]:
ssc.stop()